In [30]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv
import random
import numpy as np
import pickle

import os
import json
import pandas as pd
import copy
import random

import scipy
import scipy.optimize

import collections

In [7]:
data = []
with gzip.open('data\Grocery_and_Gourmet_Food_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

0
1143860
{'overall': 5.0, 'verified': True, 'reviewTime': '11 19, 2014', 'reviewerID': 'A1QVBUH9E1V6I8', 'asin': '4639725183', 'reviewerName': 'Jamshed Mathur', 'reviewText': 'No adverse comment.', 'summary': 'Five Stars', 'unixReviewTime': 1416355200}


In [19]:
required_vals=['overall','verified','reviewTime','reviewerID','asin','reviewText','summary','unixReviewTime']
data_cleaned=[]
cnt==0
imgcnt=0
for d in data: 
    add=True 
    for i in required_vals:
        if i not in d:
            add=False 
    if add:
        dcopy=copy.deepcopy(d) #not sure if this is really needed, will most likely remove
        if 'image' in dcopy:
            imgcnt+=1
            dcopy['hasImage']=1
        else:
            dcopy['hasImage']=0
        data_cleaned.append(dcopy)


    

In [ ]:
print(imgcnt)
#this is  small so the image feature might be useless 

In [25]:
#shuffle dataset
random.shuffle(data_cleaned)
train_data=data_cleaned[:int((len(data_cleaned)+1)*.80)]
val_data=data_cleaned[int((len(data_cleaned)+1)*.80):]

In [26]:
train_set=[]
test_set=[]
cnt=0

itemsPerUser=defaultdict(set)
usersPerItem=defaultdict(set)
items=set()
itemCount=defaultdict(int) # keep track of total times an item is reviewed

for review in train_data:
    user=review['reviewerID']
    item=review['asin']
    itemsPerUser[user].add(item)
    usersPerItem[item].add(user)
    items.add(item)
    itemCount[item]+=1
        

# See "simple bias only latent factor model"
https://cseweb.ucsd.edu//classes/fa20/cse258-a/code/workbook4.html

This is a basic latent factor model from prev assignment (needs to be improved). Uses information only from user item pairs and ratings

In [29]:
labels= [int(d['overall']) for d in train_data]
ratingMean = sum([int(d['overall']) for d in train_data]) / len(train_data)
ratingMean

4.442731190836991

In [31]:
occurrences = collections.Counter(labels)

In [44]:

N = len(train_set)
nUsers = len(itemsPerUser)
nItems = len(usersPerItem)
users = list(itemsPerUser.keys())
items = list(usersPerItem.keys())
alpha = ratingMean

userBiases = defaultdict(float)
itemBiases = defaultdict(float)
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]

def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))
    
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d[0], d[1]) for d in train_set]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(train_set)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for d in train_set:
        u,i = d[0], d[1]
        pred = prediction(u, i)
        diff = pred - int(d[2])
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return np.array(dtheta)


In [46]:

#basic  latent factor model from prev assignment (needs to be improved)
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (labels, .001))

MSE = 1.1303754596136466
MSE = 1.427133374754615
MSE = 1.1302913946672712
MSE = 1.1301753083736292
MSE = 1.1297151629412159
